# Graph and node tips

This notebook is about helpful tips or tricks you can use when working with nodes of a custom interface graph you have built.

It assumes that you have already explored the notebook on [creating custom interfaces](custom-interfaces.ipynb),
which is a pre-requisite to understanding the content here.

In [ ]:
%matplotlib widget
import plopp as pp
import scipp as sc
import numpy as np

## Node operators

<div class="versionadded" style="font-weight: bold;">

<img src="../_static/circle-exclamation.svg" width="16" height="16" />
&nbsp;
New in version 23.04.0.

</div>

We have seen in [the previous notebook](custom-interfaces.ipynb) how functions/callables wrapped in a `Node` are used to represent operations between different data sources.

It can sometimes be useful to know that nodes also have operators,
and simple operations can be implemented in a short-hand way, instead of defining functions such as `add` or `multiply`.

Consider for example

In [ ]:
a = pp.Node(np.array([1, 2, 3, 4, 5]))
b = pp.Node(np.array([6, 7, 8, 9, 10]))

The 'classical' way of adding the data from `a` and `b` would be to wrap an `add` function in a `Node` as follows:

In [ ]:
def add(x, y):
    return x + y


c = pp.Node(add, a, b)
c()

However, this short-hand is also possible:

In [ ]:
c = a + b
c()

Operators also work with normal Python objects, which simply get wrapped in a `Node`:

In [ ]:
c = a * 33.0
c()

### A short example

In [ ]:
noise = 0.5 * (np.random.random(100) - 0.5)
signal = np.sin(0.1 * np.arange(100.0)) + noise

# Noisy data
a = pp.Node(
    sc.DataArray(
        data=sc.array(dims=['time'], values=signal),
        coords={'time': sc.arange('time', 100.0, unit='s')},
    )
)
fig = pp.figure1d(a)
fig

In [ ]:
b = pp.Node(
    sc.DataArray(
        data=sc.array(dims=['time'], values=noise),
        coords={'time': sc.arange('time', 100.0, unit='s')},
    )
)

# Remove noise from signal
diff = a - b
pp.figure1d(diff)

## Using plotting functions in a graph

<div class="versionadded" style="font-weight: bold;">

<img src="../_static/circle-exclamation.svg" width="16" height="16" />
&nbsp;
New in version <TODO:VERSION>.

</div>

All the examples for working with nodes and graphs presented so far have been using the lower-level `figure1d`, `figure2d` and `figure3d` as views for the data.

These functions accept graph nodes as input, and provide visualizations for 1D, 2D or scatter 3D data.

However, say that at the end point of a graph, in the node that is providing the final result, the data still has three dimensions.
One common way of visualizing such data is as a 2D image with a slider to navigate the third dimension.

Instead of having to manually set up a slider and a node for the slicing,
it is possible to use Plopp's higher-level plotting functions directly as part of a graph.
In our present example, this would be the [slicer plot](../basics/slicer-plot.ipynb).

In [ ]:
# Make a 3D array with random values
da = pp.data.random((100, 150, 200))
da

In [ ]:
import ipywidgets as ipw
from scipp.scipy.ndimage import gaussian_filter

# Raw data root node
data_node = pp.Node(da)

# Slider to control width of smoothing kernel
slider = ipw.IntSlider(min=1, max=20, description="Smoothing")
slider_node = pp.widget_node(slider)

# Node that performs the gaussian smoothing
smooth_node = pp.Node(gaussian_filter, data_node, sigma=slider_node)

pp.show_graph(smooth_node)

In [ ]:
# Attach the `slicer` plot to the bottom node
fig = pp.slicer(smooth_node)
ipw.VBox([slider, fig])

We now have a slider at the top that controls the width of the smoothing kernel,
and a second slider at the bottom that can navigate the `z` dimension.

The final node graph is

In [ ]:
pp.show_graph(data_node)